In [ ]:
import numpy as np
import glob
import os
import pandas as pd
import scipy.signal as signal
import mne
import utils
import matplotlib.pyplot as plt

## Data pre-processing

In [ ]:
#create dataframe from CSV files
file_path= 'data/eeg_data_A/'
#eeg_dataframe_A = utils.gdf_to_df(file_path)
#eeg_dataframe_A.head()
#eeg_dataframe_A = utils.format_df(eeg_dataframe_A)
#eeg_dataframe_A.head()

### Band-pass filtering  & ICA

In [ ]:
gdf_files = glob.glob(os.path.join(file_path, '*.gdf'))


channels = 25
method = 'fastica'
raw_objects = []  # List to store raw objects

for file in gdf_files:
    raw = mne.io.read_raw_gdf(file, preload=True)
    raw.filter(l_freq=1.0, h_freq=40.0, verbose=False)
    raw_objects.append(raw)

# Concatenate raw objects
if raw_objects:
    print("concatenating raw data")
    combined_raw = mne.concatenate_raws(raw_objects)
    combined_raw.plot(duration=5, n_channels=30, title='Raw EEG Data')

                                    

[]


In [ ]:
# i ain't waiting for that again lmao
#np.save('data/filtered_eeg_array.npy', eeg_data_filtered)
"""
eeg_data_filtered = np.load('data/filtered_eeg_array.npy')

channel = 0

plt.figure(figsize=(15, 8))

# Plotting the original signal
plt.subplot(2, 1, 1)
plt.plot(eeg_data_unfiltered[channel], label='Original EEG')
plt.title('Original EEG Signal')
plt.xlabel('Time Points')
plt.ylabel('Amplitude')
plt.legend()

# Plotting the filtered signal
plt.subplot(2, 1, 2)
plt.plot(eeg_data_filtered[channel], label='Filtered EEG')
plt.title('Filtered EEG Signal')
plt.xlabel('Time Points')
plt.ylabel('Amplitude')
plt.legend()

plt.tight_layout()
plt.show()
"""

#### Independent Component Analysis

In [ ]:
channels = 25
method = 'fastica'

ica = ICA(n_components=channels, method='fastica')
ica.fit(eeg_data_filtered)
ica.plot_components()
